<span style="font-family: 'Georgia', serif; font-size: 24px; color: #2c7fb8;">
    1. Functions and Libraries
</span>

In [ ]:
from Preprocessing import *

<span style="font-family: 'Georgia', serif; font-size: 24px; color: #2c7fb8;">
    2. Load Data
</span>

<span style="font-family: 'Georgia', serif; font-size: 20px; color: #2c7fb8;">
    Transform Data: From Json to Dictionary
</span>

In [ ]:
# list to collect all teams information
teams_info = []

# Folder Path of Json Files
folder_path = r"Enter your path to the folder of Json files"

# Collect names of all the json files
for filename in os.listdir(folder_path):
    if filename.endswith('.json'):
        file_path = os.path.join(folder_path, filename)
        
        # Read the json files
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            
            # Collect track result from Sessions_List
            for session in data['Sessions_List']:
                if (session['Age_X_axis'] >= 3) and (session['Age_X_axis'] <= 90) and \
                (session['Age_Y_axis'] >= 3) and (session['Age_Y_axis'] <= 90):
                    tracks_result = [track for track in session['Track_Results']]
                    if tracks_result != []:
                        # Put the information into a list
                        team_info = {
                            'ID': session['ID'],
                            'Age_X_axis': session['Age_X_axis'],
                            'Age_Y_axis': session['Age_Y_axis'],
                            'Track_Results': tracks_result
                            }
                        teams_info.append(team_info)


print("Number of Teams")
print(len(teams_info))

<span style="font-family: 'Georgia', serif; font-size: 20px; color: #2c7fb8;">
    Extract important features and transform to Dataframe
</span>

In [ ]:
count = 0 #This is just to monitor the code
array_col = []

"""
# Excluding teams do not fit into the age group predefined
# Excluding track record that are neither complete nor made mistakes
"""


for item in teams_info:
    count = count + 1
    print(count)

    rela = determine_relationship(item['Age_X_axis'],item['Age_Y_axis']) #

    #Excluding those does not fit into the age group standard
    if rela != -1:
        for trail in item["Track_Results"]:
            
            if not trail["Completed"] and not trail["Mistake"]:# Remove these, should be restarted cases
                continue
        
            
            """
            If you don't need the trajectory data, you don't have to run this block
            
            x_aix = [x["X"] for x in trail["Points"]]
            y_aix = [y["Y"] for y in trail["Points"]]
            time_aix = [time["Time"] for time in trail["Points"]]
            
            # Interpolation and remove waiting (Run either one, depends on whether you want to remove the waiting coordinates)
            #lent_x,lent_y = remove_begin(interpolate(x_aix,time_aix),interpolate(y_aix,time_aix)) 
            #lent_x,lent_y = interpolate((x_aix,time_aix),interpolate(y_aix,time_aix))

            # Calculate the time when ball is not moving at the beginning
            #still_time = calculate_initial_still_time(lent_x,lent_y)

            # Calculate Distances
            #distance = calculate_total_distance(lent_x,lent_y)

            If you don't need the trajectory data, you don't have to run this block
            """
        
            array_col.append({"ID":item["ID"],
                              "track":trail["Track_ID"],
                              "time":trail["StartTime"],
                              "Age_group":rela,
                              #"x":lent_x,  #If you need trajectory data, also uncomment this
                              #"y":lent_y,  #If you need trajectory data, also uncomment this
                              "agex":item["Age_X_axis"],
                              "agey":item["Age_Y_axis"],
                              "time_spent":trail["Time"], # Counting the datapoints after interpolation
                              "Complete":trail["Completed"],
                              "Mistake":trail["Mistake"],
                              #"Interface":trail["Interface_Mode"],# Whether the team has turned on the interface mode
                              "Difficulty_Level":trail["Difficulty_Level"]+1,
                              #"Distance":distance
                            })



In [ ]:
df = pd.DataFrame(array_col) #117683 rows
df = df.reset_index(drop=True)

<span style="font-family: 'Georgia', serif; font-size: 24px; color: #2c7fb8;">
    3. Data Filtering
</span>

<span style="font-family: 'Georgia', serif; font-size: 20px; color: #2c7fb8;">
    Find ID with success in Level 3
</span>

In [ ]:
def find_id_Level3_success(df):
    """
    Filter the DataFrame to retain IDs that have at least one entry 
    with Difficulty_Level=3 and Complete=True, keeping all records of those IDs.
    
    Args:
        df (pd.DataFrame): Input DataFrame with columns 'ID', 'Difficulty_Level', and 'Complete'
        
    Returns:
        pd.DataFrame: Filtered DataFrame containing all records of valid IDs
    """
    # Select rows where Difficulty_Level is 3 and Complete is True
    filtered_rows = df[(df['Difficulty_Level'] == 3) & (df['Complete'] == True)]
    
    # Extract unique IDs from the filtered rows
    valid_ids = filtered_rows['ID'].unique()
    
    # Filter the original DataFrame to keep all records of valid IDs
    result = df[df['ID'].isin(valid_ids)]
    
    return result


In [ ]:
df2 = find_id_Level3_success(df)

In [ ]:
df2 = df2.reset_index(drop=True)

In [ ]:
def difficulty_3_exist(df):
    filtered_ids = df[df['Difficulty_Level'] == 3]['ID'] 
    return df[df['ID'].isin(filtered_ids)]

In [ ]:
df2 = difficulty_3_exist(df)

<span style="font-family: 'Georgia', serif; font-size: 20px; color: #2c7fb8;">
    Remove records after completing the game
</span>

In [ ]:
def remove_after_game_completed(df):
    # Sort the DataFrame by ID and time
    df_sorted = df.sort_values(['ID', 'time'])
    
    # For each ID, find the first occurrence where Complete is True and Difficulty_Level is 3
    mask = (df_sorted['Complete'] == True) & (df_sorted['Difficulty_Level'] == 3)
    first_true_indices = df_sorted[mask].groupby('ID')['time'].idxmin()
    
    # Create a mask to keep rows
    keep_mask = pd.Series(True, index=df_sorted.index)
    
    # For each ID, exclude rows after the first occurrence where Complete is True and Difficulty_Level is 3
    for id_val, idx in first_true_indices.items():
        id_rows = df_sorted.index[df_sorted['ID'] == id_val]
        rows_after = id_rows[id_rows > idx]
        keep_mask[rows_after] = False
    
    # Apply the mask and return the result
    return df_sorted[keep_mask]

In [ ]:
df3 = remove_after_game_completed(df2)
df3 = df3.reset_index(drop=True)

In [ ]:
def keep_assigned_track3(df):
    """
    Filters the DataFrame to retain only data where the track changed exactly three times in chronological order for each ID
    
    Parameters:
    df (pd.DataFrame): Input DataFrame containing columns 'track', 'ID', and 'time'
    
    Returns:
    pd.DataFrame: Filtered DataFrame
    """
    # Ensure data is sorted by ID and time
    df = df.sort_values(['ID', 'time'])
    
    # Create result DataFrame
    result_df = pd.DataFrame(columns=df.columns)
    
    # Process each ID group
    for id_value, group in df.groupby('ID'):
        track_changes = []
        current_track = None
        
        # Record each track change
        for _, row in group.iterrows():
            if row['track'] != current_track:
                current_track = row['track']
                track_changes.append(current_track)
                
            # Keep only the first three distinct tracks
            if len(track_changes) <= 3:
                result_df = pd.concat([result_df, pd.DataFrame([row])], ignore_index=True)
    
    return result_df

In [ ]:
df3 = keep_assigned_track3(df2)
df3 = df3.reset_index(drop=True)

<span style="font-family: 'Georgia', serif; font-size: 24px; color: #2c7fb8;">
    4. Feature Generation
</span>

<span style="font-family: 'Georgia', serif; font-size: 20px; color: #2c7fb8;">
    Track Feature
</span>

In [ ]:
def create_track_feature(input_df):
    """
    Create a feature dataset from the original DataFrame
    
    Parameters:
    input_df (pd.DataFrame): Original dataset containing columns ID, Difficulty_Level, and track
    
    Returns:
    pd.DataFrame: Generated feature dataset
    """
    # Create an empty DataFrame to store features
    feature_df = pd.DataFrame()
    
    # Directly retain the ID column
    feature_df['ID'] = input_df['ID'].unique()
    
    # Create a column for each difficulty level to store the corresponding track number
    for level in [1, 2, 3]:
        # Filter data for the current difficulty level and take the first track value for each ID
        level_data = input_df[input_df['Difficulty_Level'] == level].groupby('ID').first().reset_index()
        
        # Rename the column
        level_data = level_data.rename(columns={'track': f'track_level_{level}'})
        
        # Merge the results into the feature DataFrame
        feature_df = pd.merge(feature_df, level_data[['ID', f'track_level_{level}']], on='ID', how='left')
    
    return feature_df

In [ ]:
track_feature = create_track_feature(df3)

<span style="font-family: 'Georgia', serif; font-size: 20px; color: #2c7fb8;">
    Attempts Feature
</span>

In [ ]:
def create_attempts_features(df):
    """
    Generate new features from the team experiment records DataFrame.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing experiment records with columns 'ID', 'Difficulty_Level', and 'Complete'
    
    Returns:
    pd.DataFrame: DataFrame with newly generated features
    """
    # Ensure required columns exist
    required_columns = ['ID', 'Difficulty_Level', 'Complete']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Input DataFrame is missing required column: {col}")
    
    # Count rows where Difficulty_Level is 1 for each ID
    level1_count = df[df['Difficulty_Level'] == 1].groupby('ID').size().reset_index(name='Level1_Count')
    
    # Count rows where Difficulty_Level is 2 for each ID
    level2_count = df[df['Difficulty_Level'] == 2].groupby('ID').size().reset_index(name='Level2_Count')

    # Count rows where Difficulty_Level is 3 for each ID
    level3_count = df[df['Difficulty_Level'] == 3].groupby('ID').size().reset_index(name='Level3_Count')

    # Count rows where Difficulty_Level is 3 and Complete is False for each ID
    level1_incomplete_count = df[(df['Difficulty_Level'] == 1) & 
                                 (df['Complete'] == False)].groupby('ID').size().reset_index(name='Level1_Incomplete_Count')
    # Count rows where Difficulty_Level is 3 and Complete is False for each ID
    level2_incomplete_count = df[(df['Difficulty_Level'] == 2) & 
                                 (df['Complete'] == False)].groupby('ID').size().reset_index(name='Level2_Incomplete_Count')
    
    # Count rows where Difficulty_Level is 3 and Complete is False for each ID
    level3_incomplete_count = df[(df['Difficulty_Level'] == 3) & 
                                 (df['Complete'] == False)].groupby('ID').size().reset_index(name='Level3_Incomplete_Count')
    
    # Merge results
    result = level1_count
    for count_df in [level2_count, level3_count, level1_incomplete_count, level2_incomplete_count, level3_incomplete_count]:
        result = pd.merge(result, count_df, on='ID', how='left')


    # Fill NaN values with 0 (if no records match the condition for an ID)
    result = result.fillna(0)
    
    # Ensure count columns are of integer type
    for col in ['Level1_Count', 'Level2_Count', 'Level3_Count', 'Level1_Incomplete_Count', 'Level2_Incomplete_Count', 'Level3_Incomplete_Count']:
        result[col] = result[col].astype(int)
    
    return result

In [ ]:
attempts_feature = create_attempts_features(df3)

In [ ]:
# Distribution of attempts in Level 3
counts = attempts_feature['Level3_Count'].value_counts()

<span style="font-family: 'Georgia', serif; font-size: 20px; color: #2c7fb8;">
    Time Spent Feature
</span>

In [ ]:
def create_times_features(df):
    """
    Generate new features from the team experiment records DataFrame.
    
    Parameters:
    df (pd.DataFrame): DataFrame containing experiment records with columns 
                       'ID', 'Difficulty_Level', 'Complete', and 'time_spent'
    
    Returns:
    pd.DataFrame: DataFrame with newly generated features
    """
    # Ensure required columns exist
    required_columns = ['ID', 'Difficulty_Level', 'Complete', 'time_spent']
    for col in required_columns:
        if col not in df.columns:
            raise ValueError(f"Input DataFrame is missing required column: {col}")
    
    # Calculate total time_spent for Difficulty_Level 1 for each ID
    level1_time = df[df['Difficulty_Level'] == 1].groupby('ID')['time_spent'].sum().reset_index(name='Level1_Time')
    
    # Calculate total time_spent for Difficulty_Level 2 for each ID
    level2_time = df[df['Difficulty_Level'] == 2].groupby('ID')['time_spent'].sum().reset_index(name='Level2_Time')

    # Calculate total time_spent for Difficulty_Level 3 for each ID
    level3_time = df[df['Difficulty_Level'] == 3].groupby('ID')['time_spent'].sum().reset_index(name='Level3_Time')
    
    # Calculate total time_spent for Difficulty_Level 3 and Complete is False for each ID
    level3_incomplete_time = df[(df['Difficulty_Level'] == 3) & 
                               (df['Complete'] == False)].groupby('ID')['time_spent'].sum().reset_index(name='Level3_Incomplete_Time')
    
    # Merge results
    result = level1_time
    for time_df in [level2_time, level3_time, level3_incomplete_time]:
        result = pd.merge(result, time_df, on='ID', how='left')
    
    # Fill NaN values with 0 (if no records match the condition for an ID)
    result = result.fillna(0)
    
    return result

In [ ]:
time_feature = create_times_features(df3)

In [ ]:
# Create Features for the 

<span style="font-family: 'Georgia', serif; font-size: 20px; color: #2c7fb8;">
    Time Spent for Level 3 First Success
</span>

In [ ]:
def level3_first_success_time(df):
    
    filtered = df[(df['Difficulty_Level'] == 3) & (df['Complete'] == True)]
    
    result = filtered.groupby('ID').first().reset_index()
    
    result = result[['ID', 'time_spent']]

    result.rename(columns={'time_spent': 'time_spent_difficulty_3_first_success'}, inplace=True)
    
    return result

In [ ]:
level3_time_feature = level3_first_success_time(df3)

<span style="font-family: 'Georgia', serif; font-size: 20px; color: #2c7fb8;">
    Combine All Features Generated Above
</span>

In [ ]:
from functools import reduce

age_feature = df3[["ID","Age_group"]].drop_duplicates()
dataframes = [age_feature, track_feature, attempts_feature, time_feature, level3_time_feature]
merged_df = reduce(lambda left, right: pd.merge(left, right, on='ID', how='inner'), dataframes)

<span style="font-family: 'Georgia', serif; font-size: 20px; color: #2c7fb8;">
    Adding Success Rate
</span>

In [ ]:
merged_df["success_rate1"] = (merged_df["Level1_Count"]-merged_df["Level1_Incomplete_Count"])/merged_df["Level1_Count"]
merged_df["success_rate2"] = (merged_df["Level2_Count"]-merged_df["Level2_Incomplete_Count"])/merged_df["Level2_Count"]
merged_df["success_rate3"] = (merged_df["Level3_Count"]-merged_df["Level3_Incomplete_Count"])/merged_df["Level3_Count"]

In [ ]:
# Sample to validate
df[df["ID"]=="20230619-0008"]

In [ ]:
merged_df

In [ ]:
# Generate Excel File
merged_df.to_excel("Enter Your Path")